In [ ]:
#Importando os e glob para juntar arquvos csv
import os
import glob
#Importando Pandas e Numpy
import pandas as pd
import numpy as np

In [ ]:
#Em Andamento!

In [ ]:
df_concat.to_csv("Prestacao_Conta_SIGPC_Concatenada_Formatada.csv", encoding='utf-8-sig', index=False)